In [ ]:
#in case of not having this package
#! conda install pillow

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import cv2

In [2]:
data_folder = "att_faces/s"

first of all we use the first 5 pictures of each class for feature extraction using PCA . \
In other words we use the first 5 pictures for dimentionality reduction using PCA . \
Note that the dataset has 40 classes with 10 pictures so it has 400 images or instance. \
pictures' dimensions are 1024 by 1024 so we resize them to 32 by 32 to get 1024 pixels or features

In [91]:
for epoch in range(20):
    # train
    target = []
    X = np.array([],dtype=int)
    for num in range(1,41):
        #trainList = np.random.choice(range(1,11), 5, replace=False)
        trainList = np.array([1,2,3,4,5])
        for j in trainList:
            file_path = join(data_folder + str(num), str(j) + ".pgm")
            #file_path = "att_faces/s%d/%d.pgm" % (num,j)
            loaded_image=np.array(plt.imread(file_path))
            image_32x32 = cv2.resize(loaded_image, (32, 32), interpolation=cv2.IMREAD_GRAYSCALE)
            image_1024 = image_32x32.reshape(-1)
            X = np.concatenate((X,image_1024),axis=0)
            target.append(num)
    X = np.reshape(X,(200,1024))
    X = X.T #? :/

print("shape of X: ",np.shape(X))    
target = np.array(target)
y = target
print(y.shape)
        

shape of X:  (1024, 200)
(200,)


In [92]:
from sklearn.preprocessing import StandardScaler

standardized_data = StandardScaler().fit_transform(X)
print(standardized_data.shape)
print("variance of features: \n",np.var(standardized_data,axis=0))
sample_data = standardized_data
R_x = np.cov(sample_data)
print('Covariance Matrix :\n',R_x)
print('\n\n shape of Covariance Matrix :\n',R_x.shape)



(1024, 200)
variance of features: 
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
Covariance Matrix :
 [[ 0.52450121  0.51512422  0.51267137 ... -0.01585257  0.04468213
  -0.02795363]
 [ 0.51512422  0.5191006   0.51370206 ... -0.0330889   0.03507535
  -0.03385335]
 [ 0.51267137  0.51370206  0.5197142  ... -0.04723574  0.02876351
  -0.03421926]
 ...
 [-0.01585257 -0.0330889  -0.04723574 ...  1.06607211  0.7886853
   0.75657813]
 [ 0.

In [83]:
from scipy.linalg import eigh
import sys
# np.set_printoptions(threshold=sys.maxsize)
eigen_values, eigen_vectors = eigh(R_x, eigvals=(1019,1023))
eigen_values_all, eigen_vectors_all = eigh(R_x)
print(np.max(eigen_values_all))


print('Max 5 Eigen Values : \n',eigen_values)

A = eigen_vectors
# print('5 Eigen Vectors from 5 max eigen values : \n',eigen_vectors)
print('Shape of selsected Eigen Vectors : \n',A.shape)



115.54601729794814
Max 5 Eigen Values : 
 [ 25.33769153  29.47657473  49.67106486  55.00715167 115.5460173 ]
Shape of selsected Eigen Vectors : 
 (1024, 5)


In [73]:
Y = np.matmul(A.T,sample_data)

print()
print( 'shape of eigen vectors ---' , ' shape of traversed sample_data ---' , ' shape of new coordinates')
print()
print( '\t',A.shape,'\t,\t   ', sample_data.T.shape,'\t,\t', Y.shape)
print('\n',' '*20,'*'*45,'\n')
print('\t\t\t',A.shape ,' x ', sample_data.shape ,' = ', Y.shape)
print()
print(Y.shape)
cov_Y = np.cov(Y)
print(cov_Y.shape)
print(np.count_nonzero(cov_Y - np.diag(np.diagonal(cov_Y))))



shape of eigen vectors ---  shape of traversed sample_data ---  shape of new coordinates

	 (1024, 5) 	,	    (200, 1024) 	,	 (5, 200)

                      ********************************************* 

			 (1024, 200)  x  (1024, 5)  =  (5, 200)

(5, 200)
(5, 5)
20


In [67]:
new_coordinates = np.vstack((Y, y))
print(new_coordinates.shape)

(6, 200)


In [70]:
columns=("1st_principal","2nd_principal","3rd_principal","4th_principal","5th_principal","label")
dataframe = pd.DataFrame(data=new_coordinates.T, columns=columns)
print("*"*80)
print(dataframe.head(10))

********************************************************************************
   1st_principal  2nd_principal  3rd_principal  4th_principal  5th_principal  \
0       1.029389      15.102670     -11.683695       2.597424       0.635404   
1       1.441480      10.376982      -1.192030       3.581704       8.833767   
2      -4.881152      15.286837      -8.679019      -1.818384       8.001261   
3      -5.840347       1.301226     -19.089628      -1.410391       9.847155   
4      -2.282916       2.961842     -16.558806      -0.787762      12.097081   
5       1.224777      12.286186     -12.076247       3.190137       6.977436   
6       1.247818      16.535387      -8.808525       3.255922       5.254378   
7       0.439993      14.115016     -10.462126       0.437130       7.513113   
8       1.140402      16.166608      -8.608314       3.931531       6.294706   
9       1.234074      15.686049      -9.183584       4.445870       5.826574   

   label  
0    1.0  
1    1.0  
2    

till now, we implement PCA on the first 5 images of each class.
let's do it again and use 5 random images from each class instead to get better result.
then we use classifier to recognize each picture's class .
our classifier will be KNN.